### Importing Packages

In [48]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

### Splitting Data

In [50]:
#data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/gaussian_large_d_1.tex')
data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/gaussian_large_d_1.tex')
data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/gaussian_large_d_1.tex')
data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/gaussian_large_d_1.tex')v

array = np.array(data)
df = pd.DataFrame(array)

# split the dataset into training and testing data
# test_size: this is the percentage of data used for testing (20% in this case), so the rest is used for training data (80% in this case)
# random_state: this is a random number chosen that should be used each time to ensure we get the same data split each time
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:150], df.iloc[:,-1], test_size=0.2, random_state=52)

### XGBoost

In [52]:
# create model instance
# n_estimators: number of trees(estimators) the model uses --> the more used, the more accurate the model is
# max_depth: maximum depth of tree --> higher number makes model more complex, but too high can cause overfitting
# learning_rate: quantifies each tree's contribution to total prediction --> lower number takes longer, but can lead to better generalization
# objective: binary:logistic outputs probabilities. if classification is wanted, use binary:hinge
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic', enable_categorical=True)

# fit model with the training data
bst.fit(X_train, y_train)

# make predictions for the test dataset
preds = bst.predict(X_test)

# print predictions
print(preds)

# print model Accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(y_test, preds))

[1 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0
 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 0
 0 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1]
Accuracy: 0.9


### FIC

In [54]:
class FeatureClassifier:
  def __init__(self,reqAcc=0.01,classifier='DesicionTree',bias=[],control=None,n_jobs=None,random_state=None):
    self.featureClassifiers=[] #list of all the classifiers of all the selected features
    self.reqAcc=reqAcc #user specified cutoff value
    self.indexLs=[] # list of mapped index values to featureClassifiers
    self.flag=0
    self.bias=bias # list of biases for each and every label
    self.control=control #overfitting control for decision trees
    self.classifier=classifier #the classifier which is preferred by the user
    self.dic={'DecisionTree':0,'LinearRegression':1,'SVM':2,'LogisticRegression':3} #a dictionary which maps the classifier to its index
    self.n_jobs=n_jobs
    self.random_state=random_state
    self.num_lables = None

  def finIndex(self):
    #finds the index where the reqAcc condition fails and also created the indexLs[] for mapping
    for i in range(len(self.featureClassifiers)):
      if self.featureClassifiers[i][1] < self.reqAcc:
        return i
      self.indexLs.append(self.featureClassifiers[i][2])
    self.flag=1
    return i

  def fit(self,x,y):
    #applied the model to the dataset. The model is trained and saved for further prediction
    self.num_lables=len(set(y.flatten()))
    bestfeatures = SelectKBest(score_func=chi2,k=1)
    fit = bestfeatures.fit(x,y)

    for i in range(len(x[0])):
      clf=[DecisionTreeClassifier(max_depth=self.control,random_state=self.random_state),LinearRegression(n_jobs=self.n_jobs),SVC(gamma=self.control,random_state=self.random_state), LogisticRegression(penalty=self.control,random_state=self.random_state)][self.dic[self.classifier]]
      X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33,random_state=self.random_state)
      clf.fit(X_train[:,i:i+1],y_train)
      self.featureClassifiers.append((clf,fit.scores_[i],i))
    self.featureClassifiers.sort(key=lambda x:x[1],reverse=True)
    index=self.finIndex()
    if self.flag==0:
      self.featureClassifiers=self.featureClassifiers[:index]
    return

  def predict(self,x):
    #given a list of inputs, predicts the possible outputs
    if not self.bias:
      self.bias=np.zeros(self.num_lables)
    if len(self.bias)<self.num_lables:
      raise AttributeError('Please check the lenth of bias list')
    yPred=[]
    for i in range(len(x)):
      pred_arr=np.zeros(self.num_lables)
      for j in range(len(self.indexLs)):
        pred=np.round(self.featureClassifiers[j][0].predict([[x[i][self.indexLs[j]]]]))
        pred_arr[pred]+=self.featureClassifiers[j][1]+self.bias[pred[0]]
      yPred.append(np.argmax(pred_arr))
    return yPred

In [56]:
#train the model using the training sets
clf1=FeatureClassifier(0,classifier='DecisionTree',control=3)
# clf1.fit(X_train,y_train.reshape(-1,1))
clf1.fit(np.array(X_train), np.array(y_train)[:,np.newaxis].astype(int))

#predict the response for the test dataset
#model accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(np.array(y_test).astype(int),clf1.predict(np.array(X_test))))

print(clf1.indexLs)
clf1.featureClassifiers

Accuracy: 1.0
[65, 31, 123, 7, 118, 130, 46, 93, 75, 42, 142, 13, 10, 91, 9, 26, 109, 45, 44, 139, 50, 36, 92, 80, 41, 107, 99, 4, 112, 144, 43, 108, 62, 6, 17, 133, 79, 85, 83, 15, 116, 137, 141, 39, 58, 38, 135, 34, 33, 129, 59, 122, 66, 96, 84, 145, 40, 102, 11, 114, 132, 121, 125, 28, 2, 88, 136, 49, 115, 3, 1, 101, 60, 20, 140, 73, 119, 14, 8, 63, 97, 86, 138, 103, 94, 72, 68, 131, 23, 35, 70, 51, 128, 27, 29, 21, 76, 32, 53, 127, 147, 52, 87, 30, 89, 143, 105, 120, 18, 69, 126, 71, 67, 148, 100, 82, 146, 48, 98, 37, 117, 61, 74, 78, 110, 0, 77, 47, 12, 95, 106, 16, 24, 55, 90, 5, 22, 134, 54, 19, 104, 81, 57, 113, 124, 25, 111, 56, 64]


[(DecisionTreeClassifier(max_depth=3), 84.19455983035411, 65),
 (DecisionTreeClassifier(max_depth=3), 83.55354487761406, 31),
 (DecisionTreeClassifier(max_depth=3), 82.55497057799988, 123),
 (DecisionTreeClassifier(max_depth=3), 80.48054381254266, 7),
 (DecisionTreeClassifier(max_depth=3), 80.21891816406956, 118),
 (DecisionTreeClassifier(max_depth=3), 80.02467654568325, 130),
 (DecisionTreeClassifier(max_depth=3), 78.76457594546548, 46),
 (DecisionTreeClassifier(max_depth=3), 78.16002044063492, 93),
 (DecisionTreeClassifier(max_depth=3), 77.32934395062367, 75),
 (DecisionTreeClassifier(max_depth=3), 77.22917138474935, 42),
 (DecisionTreeClassifier(max_depth=3), 77.02421119238042, 142),
 (DecisionTreeClassifier(max_depth=3), 76.18680574155492, 13),
 (DecisionTreeClassifier(max_depth=3), 76.03192708502638, 10),
 (DecisionTreeClassifier(max_depth=3), 75.530579451992, 91),
 (DecisionTreeClassifier(max_depth=3), 75.37599213102858, 9),
 (DecisionTreeClassifier(max_depth=3), 74.6090292981546

### KNN

In [58]:
#create a knn classifier
#n_neighbors: predicting the label of the data point by looking at the 3 closest data points and getting them to "vote"
#algorithm: we may need to look at this if it misbehaves
neigh = KNeighborsClassifier(n_neighbors=5)

#train the model using the training sets
neigh.fit(X_train, y_train)

#predict the response for the test dataset
y_pred = neigh.predict(X_test)

#print predictions
print(y_pred)

#model accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))

[1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 0.]
Accuracy: 1.0


### SVM

In [60]:
#Create a svm Classifier
# kernel: options for kernel include linear, poly, rbf, sigmoid
    # linear: use this when data can be split by a linear function
    # poly (polynomial): use this when data can be split by a polynomial function
    # rbf (radial basis function): use this when there are clusters of one class inside another
    # sigmoid: use this when the split between classes is curved and irregular
clf = svm.SVC(kernel='linear')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# print predictions
print(y_pred)

# print model Accuracy (how often the classifier is correct)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

[1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 0.]
Accuracy: 1.0
